# 语言模型的数据集，基于周杰伦歌词

In [1]:
import sys 
sys.path.append('../')

In [2]:
import gluonbook as gb
import mxnet as mx
from mxnet import nd,gluon,init,autograd
import random
import zipfile

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
with zipfile.ZipFile('../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
print(corpus_chars[:40])

想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


## 预处理歌词，替换其中的空格和回车

In [4]:
corpus_chars = corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars = corpus_chars[:10000]

In [5]:
corpus_chars

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 如果说怀疑 可以造句如果说分离 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后在这里 不限日期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱过我的证据 让晶莹的泪滴 闪烁成回忆 伤人的美丽 你的完美主义 太彻底 让我连恨都难以下笔 将真心抽离写成日记 像是一场默剧 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后在这里 不限日期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱过我的证据 让晶莹的泪滴 闪烁成回忆 伤人的美丽 你的完美主义 太彻底 让我连恨都难以下笔 将真心抽离写成日记 像是一场默剧 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 周杰伦 周杰伦 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线乘著风 游荡在蓝天边 一片云掉落在我面前 捏成你的形

## 建立字符索引

In [6]:
idx2chars = list(set(corpus_chars))
chars2idx = dict([(char,i) for i,char in enumerate(idx2chars)])
vocab_size = len(chars2idx)
vocab_size

1027

## 将训练集中的字符都转换成索引

In [7]:
corpus_indices = [chars2idx[char] for char in corpus_chars]
sample = corpus_indices[:40]
print(sample)

[915, 754, 822, 861, 80, 874, 79, 915, 754, 700, 247, 264, 864, 519, 656, 477, 79, 915, 754, 700, 247, 545, 450, 82, 343, 257, 79, 545, 450, 82, 519, 656, 793, 79, 988, 361, 199, 361, 199, 361]


## 时序数据的采样
+ 时序数据的一个样本通常需要包含连续的字符。假设时间步数为5，样本序列为5个字符。该样本的标签序列为这些字符分别在训练集中的下一个字符

## 随机采样
+ 每次从数据中随机采样一个小批量，由于随机采样中，相邻的两个随机小批量样本的数据可能并不连续，所以每次训练我们需要重写初始化隐藏层状态

In [8]:
def data_iter_random(corpus_indices,batch_size,num_steps,ctx=None):
    num_examples = (len((corpus_indices))-1)//num_steps
    num_epochs = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    #进行随机采样
    for i in range(num_epochs):
        #小批量样本的起点
        i = i*batch_size
        batch_indice = example_indices[i:i+batch_size]
        X = [_data(pos*num_steps) for pos in batch_indice]
        Y = [_data(pos*num_steps+1) for pos in batch_indice]
        yield nd.array(X,ctx=ctx),nd.array(Y,ctx=ctx)



In [9]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  
[[12. 13. 14. 15. 16. 17.]
 [ 6.  7.  8.  9. 10. 11.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[13. 14. 15. 16. 17. 18.]
 [ 7.  8.  9. 10. 11. 12.]]
<NDArray 2x6 @cpu(0)> 

X:  
[[ 0.  1.  2.  3.  4.  5.]
 [18. 19. 20. 21. 22. 23.]]
<NDArray 2x6 @cpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [19. 20. 21. 22. 23. 24.]]
<NDArray 2x6 @cpu(0)> 



## 相邻采样

In [15]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,ctx=None):
    corpus_indices = nd.array(corpus_indices,ctx=ctx)
    data_len = len(corpus_indices)   
    batch_len = data_len//batch_size   #每个batch的样本数
    #qiepain
    indices = corpus_indices[0:batch_len*batch_size].reshape(
                                shape=(batch_size,batch_len))
    
    epoch_size = (batch_len-1) // num_steps
    for i in range(epoch_size):
        i = i*num_steps
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+num_steps+1]
        yield X,Y
    

In [16]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6,ctx=mx.gpu()):
    print('X: ', X, '\nY:', Y, '\n')

X:  
[[ 0.  1.  2.  3.  4.  5.]
 [15. 16. 17. 18. 19. 20.]]
<NDArray 2x6 @gpu(0)> 
Y: 
[[ 1.  2.  3.  4.  5.  6.]
 [16. 17. 18. 19. 20. 21.]]
<NDArray 2x6 @gpu(0)> 

X:  
[[ 6.  7.  8.  9. 10. 11.]
 [21. 22. 23. 24. 25. 26.]]
<NDArray 2x6 @gpu(0)> 
Y: 
[[ 7.  8.  9. 10. 11. 12.]
 [22. 23. 24. 25. 26. 27.]]
<NDArray 2x6 @gpu(0)> 

